# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [2]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [15]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in Psichology.'}
]
print(return_OAIResponse("Que es el EMDR", context_user))

El EMDR (Eye Movement Desensitization and Reprocessing) es una terapia utilizada principalmente para tratar el trastorno de estrés postraumático (TEPT) y otras condiciones relacionadas con traumas emocionales. Fue desarrollada por la psicóloga Francine Shapiro en la década de 1980. El EMDR se basa en la idea de que los problemas emocionales están vinculados a memorias traumáticas mal procesadas, y utiliza movimientos oculares, pulsos táctiles o sonidos bilaterales para ayudar al cerebro a procesar esas memorias de una manera más adaptativa. La terapia EMDR suele ser de corta duración en comparación con otras terapias tradicionales, y busca ayudar a los individuos a superar los síntomas de estrés postraumático y otros problemas emocionales derivados de eventos traumáticos.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [16]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in Psichology.

    Que es el apego"""}
]
print(return_OAIResponse("que es el apego?", context_user))

El apego es un vínculo emocional que se establece entre dos personas, generalmente entre un bebé y su cuidador principal, pero también puede existir entre adultos. Este vínculo se caracteriza por la necesidad de cercanía, protección, seguridad y afecto. El apego se forma a través de interacciones repetidas y significativas entre las personas, y puede influir en la forma en que una persona se relaciona con los demás en el futuro.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [17]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in Psichology.

     Que es el apego

     cuales son los tipos de apego?"""}
]
print(return_OAIResponse("nombra los tipos de apego", context_user))

Los tipos de apego son:

1. Apego seguro: Las personas con este tipo de apego suelen sentirse cómodas con la intimidad y la cercanía en sus relaciones, confían en los demás y en sí mismas, y buscan apoyo emocional cuando lo necesitan.

2. Apego ansioso-ambivalente: Las personas con este tipo de apego suelen preocuparse por la falta de cercanía en sus relaciones, tienen miedo al abandono y pueden ser emocionalmente dependientes de los demás.

3. Apego evitativo: Las personas con este tipo de apego tienden a ser reacias a la intimidad emocional, les cuesta confiar en los demás y suelen preferir la independencia y la auto suficiencia.

4. Apego desorganizado o desorientado: Las personas con este tipo de apego muestran patrones de comportamiento contradictorios y confusos en sus relaciones, a menudo experimentan miedo, confusión y ambivalencia en sus interacciones con los demás.


In [18]:
print(return_OAIResponse("que es el narcisismo?", context_user))

El narcisismo es un trastorno de la personalidad caracterizado por una excesiva preocupación por uno mismo, falta de empatía hacia los demás, necesidad constante de admiración y una percepción exagerada de su propio talento y éxito. Las personas con narcisismo tienden a actuar de manera arrogante, manipuladora y pueden tener dificultades en las relaciones interpersonales debido a su falta de consideración por los sentimientos de los demás. Este trastorno puede tener un impacto significativo en la vida de la persona que lo padece, así como en las personas que interactúan con ellos. Es importante buscar ayuda profesional si se sospecha que alguien pueda tener narcisismo para abordar adecuadamente sus causas y consecuencias.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [19]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are an in Psichology.\n\n'},
    {'role':'user', 'content':'Que es el apego?'},
    {'role':'assistant', 'content':"""El apego es un vínculo emocional que se establece entre dos personas, generalmente entre un bebé y su cuidador principal, pero también puede existir entre adultos. Este vínculo se caracteriza por la necesidad de cercanía, protección, seguridad y afecto. El apego se forma a través de interacciones repetidas y significativas entre las personas, y puede influir en la forma en que una persona se relaciona con los demás en el futuro."""},
    {'role':'user', 'content':'nombra los tipos de apego'},
    {'role':'assistant', 'content':"""Los tipos de apego son:

1. Apego seguro: Las personas con este tipo de apego suelen sentirse cómodas con la intimidad y la cercanía en sus relaciones, confían en los demás y en sí mismas, y buscan apoyo emocional cuando lo necesitan.

2. Apego ansioso-ambivalente: Las personas con este tipo de apego suelen preocuparse por la falta de cercanía en sus relaciones, tienen miedo al abandono y pueden ser emocionalmente dependientes de los demás.

3. Apego evitativo: Las personas con este tipo de apego tienden a ser reacias a la intimidad emocional, les cuesta confiar en los demás y suelen preferir la independencia y la auto suficiencia.

4. Apego desorganizado o desorientado: Las personas con este tipo de apego muestran patrones de comportamiento contradictorios y confusos en sus relaciones, a menudo experimentan miedo, confusión y ambivalencia en sus interacciones con los demás."""},
]

print(return_OAIResponse("Tipos de apego:", context_user))

Los tipos de apego son:

1. Apego seguro
2. Apego ansioso-ambivalente
3. Apego evitativo
4. Apego desorganizado o desorientado


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [20]:
context_user = [
    {'role':'system', 'content':"""You are and expert in Psychology.
    You are going to answer the question of the user explaining what apego is and providing 3 recommendations:
    Recommendations"""
    }
]

print(return_OAIResponse("Que es el apego?", context_user))

El apego es un vínculo emocional profundo que se forma entre un niño y sus cuidadores principales, generalmente los padres. Este vínculo se desarrolla a través de interacciones repetidas y afectuosas entre el niño y sus cuidadores, creando un sentido de seguridad, confianza y bienestar en el niño.

Recomendaciones para fomentar un apego seguro con tu hijo:
1. Establecer rutinas y límites claros: Los niños se sienten seguros cuando saben qué esperar y cuáles son las reglas. Mantener rutinas predecibles y establecer límites firmes pero amorosos ayuda a fortalecer el apego.
2. Demostrar afecto y atención: Los gestos de cariño, la escucha activa y la disponibilidad emocional son fundamentales para fortalecer el vínculo de apego con tu hijo. Dedica tiempo de calidad a estar presente emocionalmente con tu hijo.
3. Fomentar la comunicación y el diálogo: Hablar con tu hijo, escuchar sus emociones, preocupaciones y necesidades, y responder con empatía y comprensión contribuye a fortalecer el ap

In [21]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Tipos de apego y cual es el mejor de todos."""}
]
print(return_OAIResponse("Tipos de apego y cual es el mas sano?", context_user))

Existen diferentes tipos de apego según la teoría del apego de John Bowlby y Mary Ainsworth. Los cuatro tipos principales de apego son:

1. Apego seguro: Los niños con un apego seguro se sienten cómodos explorando su entorno cuando sus cuidadores están presentes, y buscan consuelo y seguridad en ellos cuando están angustiados. Estos niños confían en sus cuidadores y se sienten seguros en su relación.

2. Apego ansioso-ambivalente: Los niños con este tipo de apego suelen mostrar ansiedad y preocupación por la separación de sus cuidadores, pero al mismo tiempo pueden mostrarse ambivalentes respecto a recibir consuelo o rechazarlo.

3. Apego evitativo: Los niños con un apego evitativo tienden a evitar buscar consuelo o apoyo de sus cuidadores, prefiriendo explorar el entorno por su cuenta. Pueden parecer independientes, pero en realidad pueden tener dificultades para expresar sus necesidades emocionales.

4. Apego desorganizado: Este tipo de apego se caracteriza por una combinación de com

Few Shots for classification.


In [22]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I will keep it for me, because I like it. Im not going to change at anytime", context_user))

Sentiment: Positive


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

I conducted a prompt engineering exercise using OpenAI 3.5, experimenting with different techniques: zero-shot, one-shot, and few-shot prompts. Initially, I tested the bot with questions about Formula 1, and it answered most of them accurately. For example, when asked about the 2020 and 2019 Formula 1 winners, it provided correct and concrete responses based on its training data. However, when I asked about the 2024 Formula 1 winner, it couldn’t provide an answer, stating that it only had knowledge up to 2021. This demonstrated the importance of understanding the model's training cutoff date when crafting prompts for future-based topics.

Later, I switched the prompts to "You are an expert in psychology," and tested it with complex topics such as attachment theory, narcissism, and EMDR. The chatbot consistently delivered detailed and insightful responses, displaying a strong grasp of psychological concepts. Despite switching between zero-shot, one-shot, and few-shot methods, I noticed no significant issues with hallucinations or inaccuracies. The model maintained clear, factual, and focused answers throughout.

Interestingly, I didn’t observe any hallucinations during this exercise. Whether answering questions about past events or abstract psychological theories, the chatbot's responses were reliable and concrete. This highlights how well the model can perform when tasked with answering within the bounds of its training knowledge.

